In [162]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.precision", 2)
products=pd.read_csv("products.csv")
sales=pd.read_csv("sales.csv")
products=products.drop("Unnamed: 0",axis=1)
sales=sales.drop("Unnamed: 0",axis=1)
temp=sales[["invoice_date","total_sale"]]
sales["invoice_date"]=pd.to_datetime(sales["invoice_date"])
sales["week"]=pd.DatetimeIndex(sales["invoice_date"]).week
sales["month"]=pd.DatetimeIndex(sales["invoice_date"]).month
sales["year"]=pd.DatetimeIndex(sales["invoice_date"]).year
sales["weekday"]=pd.DatetimeIndex(sales["invoice_date"]).day_of_week

C:\Users\giorg\AppData\Local\Temp\ipykernel_17224\2299081029.py:11: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  sales["week"]=pd.DatetimeIndex(sales["invoice_date"]).week


In [163]:
# Get a list of products that are above the 75th pctl in terms of
# volume sold
temp=sales.groupby(by="ASIN").sum().reset_index()
sales100=temp.loc[temp["total_sale"]>6.61e+04]
sales100=sales100.drop(["InvoiceNo", "Quantity", "price", 
        "invoice_time", "CustomerID"], axis=1)
# Get a list of products that are above the 75th pctl in terms of 
# total revenue
temp=sales.groupby(by="ASIN").sum().reset_index()
quantity100=temp.loc[temp["Quantity"]>1336]
quantity100=quantity100.drop(["InvoiceNo", "price", "total_sale", 
                "invoice_time", "CustomerID"],
                axis=1)
# Find the common items between the two lists
top100=pd.merge(sales100,quantity100,on="ASIN")

In [196]:
# Let's calculate the customer's attrition on a monthly basis
temp=sales.loc[sales["year"]==2019]
temp["Customers"]=1
temp=temp.groupby("invoice_date").sum()
temp.reset_index(inplace=True)
a=list(temp.columns)
a=[i for i in a if i not in ("invoice_date","Customers")]
temp.drop(a, axis=1, inplace=True)
temp["month"]=pd.DatetimeIndex(temp["invoice_date"]).month
temp["month_end"]=pd.DatetimeIndex(temp["invoice_date"]).is_month_end
temp["month_beg"]=pd.DatetimeIndex(temp["invoice_date"]).is_month_start
temp=temp.loc[(temp["month_end"]) | (temp["month_beg"])]
temp=temp.loc[temp.duplicated(subset="month",keep=False),:]
temp.reset_index(inplace=True)


C:\Users\giorg\AppData\Local\Temp\ipykernel_17224\1525132344.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["Customers"]=1


In [165]:
attrition=pd.DataFrame()
for i in range(1,13):
    attrition=(temp["Customers"].loc[(temp["month"]==i) 
    & temp["month_end"]]
    -temp["Customers"].loc[(temp["month"]==i) 
    & temp["month_start"]])
    

In [197]:
temp

,index,invoice_date,Customers,month,month_end,month_beg
0,24,2019-02-01,1202,2,False,True
1,47,2019-02-28,946,2,True,False
2,48,2019-03-01,1075,3,False,True
3,74,2019-03-31,1002,3,True,False
4,96,2019-05-01,441,5,False,True
5,120,2019-05-31,883,5,True,False
6,121,2019-06-01,684,6,False,True
7,146,2019-06-30,1166,6,True,False
8,147,2019-07-01,808,7,False,True
9,172,2019-07-31,1231,7,True,False


In [195]:
for i in range(0,17,2):
    print(i)

0
2
4
6
8
10
12
14
16


In [167]:
# Find the revenue per customer for each country 
temp=sales.copy(deep=True)
temp["ca"]=1
temp=temp.groupby(by="Country").sum()
a=list(temp.columns)
a=[i for i in a if i not in ("total_sale","ca")]
temp.drop(a,axis=1,inplace=True)
temp["Revenue per customer"]=temp["total_sale"]/temp["ca"]
temp.rename({"total_sale":"Total Sales", "ca":"Customers"},
            axis=1, inplace=True)
temp.sort_values(by="Revenue per customer",ascending=False)

,Total Sales,Customers,Revenue per customer
Country,,,
Sweden,4.35e+06,426,10219.99
Japan,3.05e+06,319,9555.48
Netherlands,2.17e+07,2316,9366.28
Australia,9.76e+06,1243,7850.77
Czech Republic,8.82e+04,24,3674.50
Singapore,7.36e+05,217,3393.04
Ireland,1.50e+07,7186,2087.75
Israel,4.64e+05,240,1933.76
Denmark,6.85e+05,382,1793.97


In [171]:
# Calculate average order size and revenue for each country
temp=sales.copy(deep=True)
temp["Items"]=1
temp=temp.groupby(by=["Country","InvoiceNo"]).sum()
temp.reset_index(inplace=True)
temp.rename({"weekday":"no_inv"},axis=1,inplace=True)
temp["no_inv"]=1
temp=temp.groupby(by="Country").sum()
temp["Items per order"]=temp["Items"]/temp["no_inv"]
temp["Revenue per order"]=temp["total_sale"]/temp["no_inv"]
temp["Revenue per item"]=temp["Revenue per order"]/temp["Items"]
a=list(temp.columns)
a=[i for i in a if i not in ("Items per order", 
    "Revenue per order", "Revenue per item")]
temp.drop(a,axis=1,inplace=True)
temp.sort_values(by="Revenue per order", ascending=False)

,Items per order,Revenue per order,Revenue per item
Country,,,
Netherlands,24.64,230769.21,99.64
Australia,21.07,165398.46,133.06
Japan,16.79,160431.41,502.92
Sweden,12.53,128050.44,300.59
Singapore,31.00,105184.22,484.72
Israel,48.00,92820.54,386.75
Ireland,27.64,57702.27,8.03
Switzerland,36.85,55264.00,31.24
United States,35.80,53704.78,300.03


In [ ]:
temp=sales.groupby(by=["Country","InvoiceNo"]).sum()
temp.reset_index(inplace=True)
temp

In [ ]:
fig,ax=plt.subplots(figsize=(5,5),facecolor="snow")
temp=products["product_type"].value_counts()
label=[i for i in temp.index]
label=[i.capitalize() for i in label]
temp.plot.pie(
        labels=label, autopct="%1.0f%%")
ax.set_ylabel("")

In [ ]:
# Let's plot the total sales for each day of the week
# Interestingly enough, there are no orders on Tuesdays. 
# That can not be correct, so we assume it's due to date corruption 
temp=sales.groupby(by="weekday").sum()
a=list(temp.columns)
a=[i for i in a if i not in ("total_sale")]
temp.drop(a, axis=1, inplace=True)
temp.loc[1]=0
temp.reset_index(inplace=True)

fig,ax=plt.subplots(figsize=(12,5))
sns.lineplot(data=temp, x="weekday", y="total_sale",
            color="crimson")
plt.xticks([0, 1, 2, 3, 4, 5, 6], ["Monday", "Tuesday", "Wednesday",
            "Thursday", "Friday", "Saturday", "Sunday"])
plt.title("Sales distribution on the days of the week")
plt.ylabel("Sales in 100s of millions")
plt.xlabel("Day of the week")

In [ ]:
# Do it again but with better grouping and rolling averages
fig,ax=plt.subplots(2,1,figsize=(17,8.5),sharex=True)
temp=pd.merge(sales,products,on="ASIN")
temp=temp.groupby(by="invoice_date").sum()
temp.reset_index(inplace=True)
temp["avg"]=(temp["total_sale"].rolling(7).sum())
sns.lineplot(ax=ax[0],x=temp["invoice_date"], y=temp["avg"],
        color="crimson")
temp["avg"]=(temp["total_sale"].rolling(30).sum())        
sns.lineplot(ax=ax[1],x=temp["invoice_date"], y=temp["avg"],
        color="crimson")        
ax[0].set(title="Weekly average revenue", xlabel="", 
        ylabel="Revenue in 10s of millions")
ax[1].set(title="Monthly average revenue", xlabel="Date", 
        ylabel="Revenue in 10s of millions")

In [ ]:
# Daily sales throughout the year, with Black Friday denoted by vertical line
fig,ax=plt.subplots(figsize=(17,8.5))
temp=sales
temp=temp.groupby(by="invoice_date").sum()
temp.reset_index(inplace=True)
sns.lineplot(x=temp["invoice_date"], y=temp["total_sale"],
        color="crimson")
lst=temp["invoice_date"].loc[temp["invoice_date"]=="2019-11-28"]
plt.axvline(lst)


In [ ]:
fig,ax=plt.subplots(figsize=(12,5))
temp=pd.merge(sales,products,on="ASIN")
temp=temp.groupby(by=["week","product_type"]).sum()
temp.reset_index(inplace=True)
sns.lineplot(x=temp["week"], y=temp["Quantity"],
        hue=temp["product_type"],
        color="darkslategrey")
plt.title("Sales breakdown based on product type")
plt.xlabel("Date")


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
temp=sales.loc[sales["year"]==2019]
temp=temp[["month","total_sale"]]
temp.set_index("month", inplace=True)
temp.index=pd.to_datetime(temp.index)
dec=seasonal_decompose(temp["total_sale"], model="additive",period=12)
dec.trend.plot()

In [ ]:
# Attempt at web scrapping for accurate ratings and up to date prices
# WIP
from bs4 import BeautifulSoup
import requests
site="https://www.amazon.de/dp/B00000JRRD"
head=({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/18.17763"})
r=requests.get(url=site,headers=head)
soup=BeautifulSoup(r.content, "lxml")
title=soup.find("span", attrs={"class":"a-price-whole"})